In [1]:
import re
import os
import pandas as pd
import numpy as np

In [8]:
def extract_phi_nr(file_path):
    import os
    import re
    import pandas as pd
    import numpy as np

    # ===============================
    # 2. LOAD TABLE
    # ===============================
    ext = os.path.splitext(file_path.lower())[1]

    if ext == ".csv":
        df = pd.read_csv(file_path)
    elif ext == ".tsv":
        df = pd.read_csv(file_path, sep="\t")
    else:
        df = pd.read_csv(file_path, delim_whitespace=True)

    # ===============================
    # 3. FIND SIPM COLUMNS
    # ===============================
    phi_prefix = "phi_nr_sipm"
    unc_prefix = "uncertainty_phi_nr_sipm"  # kept only for extraction, not used

    phi_cols = [c for c in df.columns if c.startswith(phi_prefix)]

    sipm_ids = []
    for c in phi_cols:
        m = re.match(rf"{phi_prefix}(\d+)", c)
        if m:
            sipm_ids.append(int(m.group(1)))
        else:
            pass

    # ===============================
    # 4. STACK ALL ROWS & SIPMS
    # ===============================
    rows = []
    for sid, phi_col in zip(sipm_ids, phi_cols):
        unc_col = f"{unc_prefix}{sid}"
        if unc_col not in df.columns:
            raise ValueError(f"Missing column: {unc_col}")

        tmp = df[[phi_col, unc_col]].copy()
        tmp.columns = ["phi", "unc"]
        tmp["sipm"] = f"sipm{sid}"
        rows.append(tmp)

    long = pd.concat(rows, ignore_index=True).dropna(subset=["phi"])
    long["phi"] = long["phi"].astype(float)

    # ===============================
    # 5. GLOBAL (UNWEIGHTED) MEAN
    # ===============================
    phi_mean = long["phi"].mean()

    # ===============================
    # 6. RELATIVE DISTANCE (UNWEIGHTED)
    # ===============================
    long["relative"] = (long["phi"] - phi_mean) / phi_mean

    # ===============================
    # 7. ONE RESULT PER SIPM (UNWEIGHTED)
    # ===============================
    per_sipm = (
        long.groupby("sipm", as_index=False)["relative"]
        .mean()
        .rename(columns={"relative": "mean_relative_distance"})
    )

    # ===============================
    # 7.5 CENTER so OUTPUT SUMS TO 0
    # ===============================
    per_sipm["mean_relative_distance"] = (
        per_sipm["mean_relative_distance"] - per_sipm["mean_relative_distance"].mean()
    )
    # now sum(per_sipm["mean_relative_distance"]) == 0 (up to floating error)

    # ===============================
    # 8. NORMALIZATION: (|min| + |max|)/2 = 1
    # ===============================
    rmin = per_sipm["mean_relative_distance"].min()
    rmax = per_sipm["mean_relative_distance"].max()
    scale = (abs(rmin) + abs(rmax)) / 2.0

    per_sipm["normalized_relative_distance"] = (
        per_sipm["mean_relative_distance"] / scale if scale != 0 else 0.0
    )

    # ===============================
    # 9. SORT
    # ===============================
    per_sipm["sipm_num"] = per_sipm["sipm"].str.extract(r"(\d+)").astype(int)
    per_sipm = per_sipm.sort_values("sipm_num").drop(columns=["sipm_num", "mean_relative_distance"])

    # ===============================
    # 10. RETURN (SUM SHOULD BE 0)
    # ===============================
    out = per_sipm["normalized_relative_distance"].to_numpy()

    # Optional: enforce exact zero-sum (tiny floating fix)
    out = out - out.mean()

    return out


In [9]:
w1 = extract_phi_nr(r"C:\ACME_analysis\multiple_results\sequencesipm_results\0016.2143.csv")

In [10]:
w2 = extract_phi_nr(r"C:\ACME_analysis\multiple_results\sequencesipm_results\0016.2128_0016.2129_0016.2130_0016.2131_0016.2132.csv")

In [11]:
w3 = extract_phi_nr(r"C:\ACME_analysis\multiple_results\sequencesipm_results\0016.2118_0016.2119_0016.2120_0016.2121_0016.2122_.csv")

In [12]:
w4 = extract_phi_nr(r"C:\ACME_analysis\multiple_results\sequencesipm_results\0016.1978_0016.1979_0016.1980_0016.1981_0016.1982.csv")

In [ ]:
w5 = extract_phi_nr(r"C:\ACME_analysis\multiple_results\sequencesipm_results\0016.2165_0016.2166_0016.2167_0016.2168_0016.2169_.csv")


In [13]:
print(w1)
print(w2)
print(w3)
print(w4)

[ 0.90604702  1.1688708   0.18140322 -0.67920187 -0.26553848 -0.8311292
 -0.23242161 -0.24802988]
[ 0.98219765  1.00975402  0.01531774 -0.51662425 -0.0564525  -0.99024598
 -0.189791   -0.25415569]
[ 0.5950279   1.05272236 -0.13049754 -0.11931037  0.1441676  -0.94727764
 -0.20402445 -0.39080786]
[ 0.84985907  0.97576655 -0.02996912 -1.02423345  0.27421014 -0.27956448
 -0.3547034  -0.41136532]
